In [1]:
!pip install redis

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Análise com Redis

Este notebook contem as aplicações práticas das estruturas de dados probabilísticas do Redis, foi escolhido dentre elas o **Bloom Filter** e **HyperLogLog**, para otimizar consultas e estimar contagens em um grande conjunto de dados.

#### Carrego o dataset

In [7]:
import pandas as pd

parquet_path = "../data/dataset/data_final_optimized.parquet"
data = pd.read_parquet(parquet_path)

print("✅ Carregamento concluído.")

Iniciando carregamento ultrarrápido do Parquet...
✅ Carregamento concluído. O DataFrame está pronto para uso.


# Bloom Filter

Para a análise com Bloom Filter, decidi focar nas estações que registraram temperaturas extremamente altas (acima de 40°C).
Primeiro, crio uma chave específica para “calor extremo” no banco e pré-carrego o filtro com todas as estações que excederam esse limite.

Após a inserção dos dados, realizo consultas para verificar rapidamente se uma determinada estação já registrou temperaturas elevadas, utilizando o Bloom Filter como mecanismo probabilístico de checagem.

## Testando a conexão com o redis

In [2]:
import redis

r = redis.Redis(
    host='redis-13200.crce216.sa-east-1-2.ec2.cloud.redislabs.com',
    port=13200,
    decode_responses=True,
    username="default",
    password="o3lq21mCTKPTi4BaMmbKzhBM8XKTCeZh",
)

try:
    r.ping()
    print("✅ Conectado ao Redis Cloud com sucesso!")
except redis.ConnectionError as e:
    print(f"❌ Erro de conexão: {e}")
    exit()

✅ Conectado ao Redis Cloud com sucesso!


## Realizando a exportação das estações

Filtro os dados que irão ser importados para o redis e os importo em batchs para não sobrecarregar o banco

In [ ]:
print("\n--- Iniciando Bloom Filter ---")
KEY_BLOOM = "bf:calor_extremo"
LIMIAR_CALOR = 400 # 40.0 graus Celsius

#Filtro para evitar de enviar todas as estações
df_calor = data[(data['element'] == 'TMAX') & (sdata['value'] > LIMIAR_CALOR)]
estacoes_com_calor = df_calor['station'].unique()

print(f"Total de estações únicas com calor extremo para registrar: {len(estacoes_com_calor)}")

try:
    # Estimo um erro de 1%
    r.bf().reserve(KEY_BLOOM, 0.01, 100000)
    print("Filtro Bloom criado.")
except redis.exceptions.ResponseError as e:
    print(f"Nota: {e} (provavelmente o filtro já existe ou módulo ausente)")

# Inserção em batch
if len(estacoes_com_calor) > 0:
    pipe = r.pipeline()
    
    for i, estacao in enumerate(estacoes_com_calor):
        pipe.bf().add(KEY_BLOOM, estacao)
        if (i + 1) % 1000 == 0:
            pipe.execute()
            
    pipe.execute() # Executa o restante
    print("✅ Carga do Bloom Filter finalizada.")


--- Iniciando Bloom Filter ---
Total de estações únicas com calor extremo para registrar: 402
Nota: item exists (provavelmente o filtro já existe ou módulo ausente)
✅ Carga do Bloom Filter finalizada.


### Testando a consulta

Valido a consulta procurando por alguma estação que realmente tenha registrado temperaturas extremas independente da época

In [ ]:
estacao_teste = estacoes_com_calor[0] if len(estacoes_com_calor) > 0 else "AE000041196"
existe = r.bf().exists(KEY_BLOOM, estacao_teste)
print(f"Teste: A estação {estacao_teste} já registrou > 40 graus? {'Sim' if existe else 'Não'}")

Teste: A estação AE000041196 já registrou > 40 graus? Sim


# Hyper LogLog

Para a análise com o Hyper LogLog, escolhi estimar o numero de estações ativas por diferentes decádas.
Considero apenas estações que possuem um ou mais registros dentro daquela decada.

In [ ]:

print("\n--- Iniciando HyperLogLog ---")

estacoes_por_ano = data.groupby('year')['station'].unique()
pipe = r.pipeline()

for ano, estacoes in estacoes_por_ano.items():
    chave_hll = f"hll:estacoes:{ano}"
    
    chunk_size = 5000
    for i in range(0, len(estacoes), chunk_size):
        chunk = estacoes[i:i + chunk_size]
        pipe.pfadd(chave_hll, *chunk) # O asterisco desempacota a lista

pipe.execute()
print("✅ Carga do HyperLogLog finalizada.")

# Procuramos o numero de estações que tiveram registro por ano
print("\n--- Resultado: Estações Ativas por Década (HLL) ---")
decadas = [1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010]

for ano in decadas:
    chave = f"hll:estacoes:{ano}"
    contagem = r.pfcount(chave)
    print(f"Ano {ano}: ~{contagem} estações ativas")


--- Iniciando HyperLogLog ---
✅ Carga do HyperLogLog finalizada.

--- Resultado: Estações Ativas por Década (HLL) ---
Ano 1940: ~295 estações ativas
Ano 1950: ~413 estações ativas
Ano 1960: ~621 estações ativas
Ano 1970: ~614 estações ativas
Ano 1980: ~824 estações ativas
Ano 1990: ~851 estações ativas
Ano 2000: ~874 estações ativas
Ano 2010: ~914 estações ativas
